# ** 07.Regularized Model-Ridge Code 실습**

[목적]
  1. Linear Regression
    - 단순 Linear Regression을 활용하여 변수의 중요도 및 방향성을 알아봄
    - 매우 심플한 모델이기 때문에 사이즈가 큰 데이터에 적합하지 않음
    - 하지만 설명력에서는 큰 장점이 있음
  2. Ridge Regression
    - Regularized Linear Model을 활용하여 Overfitting을 방지함
    - Hyperparameter lamba를 튜닝할 때 for loop 뿐만 아니라 GridsearchCV를 통해 돌출해봄
  3. Regularized Linear Models의 경우 X's Scaling을 필수적으로 진행해야함

[Process]
  1. Define X's & Y
  2. Split Train & Valid dataset
  3. Modeling
  4. Model 해석

In [1]:
# Package
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Sklearn Toy Data
from sklearn.datasets import load_diabetes

In [2]:
# Data Loading (당뇨병)
data = load_diabetes()

In [3]:
print('>>>>X Shape : {}'.format(data['data'].shape))
print('>>>>Y Shape : {}'.format(data['target'].shape))

>>>>X Shape : (442, 10)
>>>>Y Shape : (442,)


In [4]:
# 데이터 설명
print(data['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [6]:
pd.DataFrame(data['data'], columns=data['feature_names']).describe().applymap(lambda x: f"{x:0.2f}")

age     sex     bmi      bp      s1      s2      s3      s4      s5  \
count  442.00  442.00  442.00  442.00  442.00  442.00  442.00  442.00  442.00   
mean    -0.00    0.00   -0.00   -0.00   -0.00    0.00   -0.00   -0.00    0.00   
std      0.05    0.05    0.05    0.05    0.05    0.05    0.05    0.05    0.05   
min     -0.11   -0.04   -0.09   -0.11   -0.13   -0.12   -0.10   -0.08   -0.13   
25%     -0.04   -0.04   -0.03   -0.04   -0.03   -0.03   -0.04   -0.04   -0.03   
50%      0.01   -0.04   -0.01   -0.01   -0.00   -0.00   -0.01   -0.00   -0.00   
75%      0.04    0.05    0.03    0.04    0.03    0.03    0.03    0.03    0.03   
max      0.11    0.05    0.17    0.13    0.15    0.20    0.18    0.19    0.13   

           s6  
count  442.00  
mean     0.00  
std      0.05  
min     -0.14  
25%     -0.03  
50%     -0.00  
75%      0.03  
max      0.14

In [7]:
data = pd.read_csv("https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt", sep='\t')

In [8]:
data.describe()

AGE         SEX         BMI          BP          S1          S2  \
count  442.000000  442.000000  442.000000  442.000000  442.000000  442.000000   
mean    48.518100    1.468326   26.375792   94.647014  189.140271  115.439140   
std     13.109028    0.499561    4.418122   13.831283   34.608052   30.413081   
min     19.000000    1.000000   18.000000   62.000000   97.000000   41.600000   
25%     38.250000    1.000000   23.200000   84.000000  164.250000   96.050000   
50%     50.000000    1.000000   25.700000   93.000000  186.000000  113.000000   
75%     59.000000    2.000000   29.275000  105.000000  209.750000  134.500000   
max     79.000000    2.000000   42.200000  133.000000  301.000000  242.400000   

               S3          S4          S5          S6           Y  
count  442.000000  442.000000  442.000000  442.000000  442.000000  
mean    49.788462    4.070249    4.641411   91.260181  152.133484  
std     12.934202    1.290450    0.522391   11.496335   77.093005  
min     22.000000    2.000000    3.258100   58.000000   25.000000  
25%     40.250000    3.000000    4.276700   83.250000   87.000000  
50%     48.000000    4.000000    4.620050   91.000000  140.500000  
75%     57.750000    5.000000    4.997200   98.000000  211.500000  
max     99.000000    9.090000    6.107000  124.000000  346.000000

In [9]:
# X's & Y Split
Y = data['Y']
X = data.drop(columns=['Y'])
X = pd.get_dummies(X, columns=['SEX'])

In [10]:
X

AGE   BMI      BP   S1     S2    S3    S4      S5   S6  SEX_1  SEX_2
0     59  32.1  101.00  157   93.2  38.0  4.00  4.8598   87      0      1
1     48  21.6   87.00  183  103.2  70.0  3.00  3.8918   69      1      0
2     72  30.5   93.00  156   93.6  41.0  4.00  4.6728   85      0      1
3     24  25.3   84.00  198  131.4  40.0  5.00  4.8903   89      1      0
4     50  23.0  101.00  192  125.4  52.0  4.00  4.2905   80      1      0
..   ...   ...     ...  ...    ...   ...   ...     ...  ...    ...    ...
437   60  28.2  112.00  185  113.8  42.0  4.00  4.9836   93      0      1
438   47  24.9   75.00  225  166.0  42.0  5.00  4.4427  102      0      1
439   60  24.9   99.67  162  106.6  43.0  3.77  4.1271   95      0      1
440   36  30.0   95.00  201  125.2  42.0  4.79  5.1299   85      1      0
441   36  19.6   71.00  250  133.2  97.0  3.00  4.5951   92      1      0

[442 rows x 11 columns]

[Data Split]

- Data Split을 진행할 때 BigData의 경우 꼭 indexing을 추출하여 모델에 적용시켜야 함
- 이유는 Data Split하여 새로운 Data set을 만들 경우 메모리에 부담을 주기 때문

In [11]:
X.shape[0]

442

In [14]:
idx = list(range(X.shape[0]))
train_idx, valid_idx = train_test_split(idx, test_size=0.3, random_state=2023)
print(">>>> # of Train data : {}".format(len(train_idx)))
print(">>>> # of valid data : {}".format(len(valid_idx)))

>>>> # of Train data : 309
>>>> # of valid data : 133


In [15]:
# Linear Regression 
result = LinearRegression().fit(X.iloc[train_idx], Y.iloc[train_idx])

In [ ]:
import scipy 
from sklearn import metrics

def sse(clf, X, y):
    """Calculate the standard squared error of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The standard squared error of the model.
    """
    y_hat = clf.predict(X)
    sse = np.sum((y_hat - y) ** 2)
    return see / X.shape[0]